In [177]:
import pandas as pd
import re
from word2number import w2n

In [178]:
# This function reads the data and returns pandas dataframes
def extract():
    pizzas = pd.read_csv('ORIGINALS/pizzas.csv')
    pizza_types = pd.read_csv('ORIGINALS/pizza_types.csv', encoding='latin-1')
    orders = pd.read_csv('ORIGINALS/orders_2016.csv', sep=';')
    order_details = pd.read_csv('ORIGINALS/order_details_2016.csv', sep=';', encoding='latin-1')
    data_dictionary = pd.read_csv('ORIGINALS/data_dictionary.csv')
    return pizzas, pizza_types, orders, order_details, data_dictionary


In [179]:
# In this cell we will clean up the data and get rid of null values and transform the data to the correct format

# First, we will get rid of the null values in the orders and order_details dataframe
def clean_data(ordered_pizzas):
    ordered_pizzas.pop('time')
    ordered_pizzas.pop('order_details_id')
    ordered_pizzas = ordered_pizzas.dropna()
    ordered_pizzas = ordered_pizzas[ordered_pizzas['pizza_id'] != 'nan']
    ordered_pizzas = ordered_pizzas.reset_index(drop=True)
    return ordered_pizzas

# Then we will transform the data to the correct format
def transform_data(orders, order_details):
    ordered_pizzas = pd.merge(orders, order_details, on='order_id')
    ordered_pizzas['date'] = [control_date(x) for x in ordered_pizzas['date']]
    # We will use regex to write pizza_id with the correct format (e.g. pizza_flv_size)
    ordered_pizzas['pizza_id'] = [re.sub('[ -]', '_', str(x)) for x in ordered_pizzas['pizza_id']]
    # There are some values that are misswritten in the pizza_id column, so we will correct them
    ordered_pizzas['pizza_id'] = [re.sub('@', 'a', str(x)) for x in ordered_pizzas['pizza_id']]
    ordered_pizzas['pizza_id'] = [re.sub('3', 'e', str(x)) for x in ordered_pizzas['pizza_id']]
    ordered_pizzas['pizza_id'] = [re.sub('0', 'o', str(x)) for x in ordered_pizzas['pizza_id']]
    # We will use the word2number library to transform the string number to integer
    ordered_pizzas['quantity'] = [control_w2n(x) for x in ordered_pizzas['quantity']]
    ordered_pizzas = clean_data(ordered_pizzas)
    return ordered_pizzas.astype({'quantity': 'int64'})


def control_w2n(x):
    try:
        return w2n.word_to_num(str(x))
    except:
        return

def control_date(x):
    try:
        if re.findall('\A\d\d-\d\d-\d\d', x):
            return pd.to_datetime(x, format='%d-%m-%y %H:%M:%S')
        else:
            return pd.to_datetime(float(x), unit='s')
    except:
        return pd.to_datetime(x)

In [180]:
# In this cell we create transformed csv files from the original ones

# We create a csv with the pizzas ordered in each order, instead of having each pizza in a different row
def csv_orders(ordered_pizzas):
    pizza_order = {}
    for i in ordered_pizzas['order_id']:
        if i not in pizza_order:
            pizza_order[i] = []
    for i in range(len(ordered_pizzas)):
        for _ in range(ordered_pizzas['quantity'][i]):
            pizza_order[ordered_pizzas['order_id'][i]].append(ordered_pizzas['pizza_id'][i])
    ordered_pizzas['pizzas'] = [pizza_order[x] for x in ordered_pizzas['order_id']]
    ordered_pizzas.pop('quantity')
    ordered_pizzas.pop('pizza_id')
    ordered_pizzas = (ordered_pizzas.drop_duplicates(subset=['order_id'])).reset_index(drop=True)
    ordered_pizzas.to_csv('TRANSFORMED/ordered_pizzas_2016.csv', index=False)
    return ordered_pizzas

# This function adds to the ordered_pizzas dataframe the day of the week of each order
def csv_with_days(ordered_pizzas):
    dates = []
    days = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
    for date in ordered_pizzas['date']:
        dates.append(days[date.weekday()])
    ordered_pizzas['day'] = dates
    ordered_pizzas.to_csv('TRANSFORMED/ordered_pizzas_2016.csv', index=False)
    return ordered_pizzas

# This function creates a csv with all the ingredients and the amount of each one for each day of the week (and total week)
def csv_ingredients(pizza_types):
    all_ingredients = []
    for ingredients in pizza_types['ingredients']:
        for ingredient in ingredients.split(', '):
            if ingredient not in all_ingredients:
                all_ingredients.append(ingredient)
    ingredients_df = pd.DataFrame({'ingredient': all_ingredients, 'Monday': [0 for _ in range(len(all_ingredients))], 'Tuesday': [0 for i in range(len(all_ingredients))], 'Wednesday': [0 for i in range(len(all_ingredients))], 'Thursday': [0 for i in range(len(all_ingredients))], 'Friday': [0 for i in range(len(all_ingredients))], 'Saturday': [0 for i in range(len(all_ingredients))], 'Sunday': [0 for i in range(len(all_ingredients))], 'Total': [0 for i in range(len(all_ingredients))]})
    ingredients_df.to_csv('TRANSFORMED/ingredients_2016.csv', index=False)
    return ingredients_df


In [181]:
# We create a function that returns the pizza flavour and size of a pizza
def search_pizza(pizza_id, pizzas):
    pizza = pizzas[pizzas['pizza_id'] == pizza_id]
    return pizza['pizza_type_id'].values[0], pizza['size'].values[0]

# This function adds to the pizzas csv the amount of pizzas ordered each day of the week
# We calculate this by pizza, where we use the size as a factor 
def create_csv_with_pizzas_per_day(ordered_pizzas, pizza_types, pizzas_data, date):
    lst = [0 for _ in range(len(pizza_types))]
    weigths = {'S': 1, 'M': 1.5, 'L': 2, 'XL': 2.5, 'XXL': 3}
    pizza_counts = {'Monday': lst.copy(), 'Tuesday': lst.copy(), 'Wednesday': lst.copy(), 'Thursday': lst.copy(), 'Friday': lst.copy(), 'Saturday': lst.copy(), 'Sunday': lst.copy()}
    for _, order in ordered_pizzas.iterrows():
        if pd.to_datetime(order['date']) >= date:
            break
        else:
            day = order['day']
            pizzas = order['pizzas']
            for pizza in pizzas:
                pizza_flavour, size = search_pizza(pizza, pizzas_data)
                ind = pizza_types[pizza_types['pizza_type_id'] == pizza_flavour].index.values[0]
                pizza_counts[day][ind] += 1*weigths[size]
    for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']:
        pizza_types[day] = pizza_counts[day]
    pizza_types.to_csv('TRANSFORMED/pizza_counts_2016.csv', index=False)
    return pizza_types

# This function calculates the amount of ingredients needed a specific day of the week
def ingredients_quantity(day, pizza_types, pizza_type_id, days_difference):
    aux = (pizza_types[pizza_types['pizza_type_id'] == pizza_type_id][day].values[0])
    return aux*7/days_difference

# This function predicts the ingredients needed for the following week
def predict(pizza_types, ingredients_df, days_difference):
    for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']:
        for ingredients in pizza_types['ingredients']:
            pizza_type_id = pizza_types[pizza_types['ingredients'] == ingredients]['pizza_type_id'].values[0]
            for ingredient in ingredients.split(', '): 
                ind = ingredients_df[ingredients_df['ingredient'] == ingredient].index.values[0]   
                prediction = ingredients_quantity(day, pizza_types, pizza_type_id, days_difference)
                ingredients_df.loc[ind,[day]] += prediction
                ingredients_df.loc[ind,['Total']] += prediction
    ingredients_df.to_csv('TRANSFORMED/ingredients_2016.csv', index=False)
    return ingredients_df

# def check_ingredients(ingredients_df, pizza_types, orders, date, days_difference, pizzas_data):
#     weigths = {'S': 1, 'M': 1.5, 'L': 2, 'XL': 2.5, 'XXL': 3}
#     days = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
#     for _, order in orders.iterrows():
#         n = (pd.to_datetime(order['date'], format='%d/%m/%Y') - date).days
#         if n < 7 and n >= 0:
#             day = days[pd.to_datetime(order['date'], format='%d/%m/%Y').weekday()]
#             pizzas = order['pizzas']
#             for pizza in pizzas:
#                 pizza_flavour, size = search_pizza(pizza, pizzas_data)
#                 ingredients = pizza_types[pizza_types['pizza_type_id'] == pizza_flavour]['ingredients'].values[0]
#                 for ingredient in ingredients.split(', '): 
#                     ind = ingredients_df[ingredients_df['ingredient'] == ingredient].index.values[0]   
#                     ingredients_df.loc[ind,[day]] -= 1*weigths[size]
#                     ingredients_df.loc[ind,['Total']] -= 1*weigths[size]
#     return ingredients_df


In [182]:
date = pd.to_datetime('2016-12-31', format='%Y-%m-%d')
days_difference = (date - pd.to_datetime('2016-01-01', format='%Y-%m-%d')).days
pizzas, pizza_types, orders, order_details, data_dictionary = extract()
ordered_pizzas = transform_data(orders, order_details)
ordered_pizzas = csv_orders(ordered_pizzas)
ordered_pizzas = csv_with_days(ordered_pizzas)
ordered_pizzas = ordered_pizzas.sort_values(by=['date'])
display(ordered_pizzas)

,order_id,date,pizzas,day
13742,36,2015-12-31 23:00:00,"[classic_dlx_m, bbq_ckn_m]",Thursday
12562,29,2015-12-31 23:00:00,[green_garden_s],Thursday
6357,1,2015-12-31 23:00:00,[hawaiian_m],Thursday
16848,55,2015-12-31 23:00:00,[peppr_salami_s],Thursday
10686,27,2015-12-31 23:00:00,"[classic_dlx_m, bbq_ckn_l]",Thursday
...,...,...,...,...
12668,21315,2016-12-31 17:47:46,"[peppr_salami_s, sicilian_l]",Saturday
14515,21318,2016-12-31 18:07:03,"[pepperoni_s, pep_msh_pep_m, bbq_ckn_m]",Saturday
4475,21319,2016-12-31 18:07:19,"[prsc_argla_m, veggie_veg_s]",Saturday
2533,21321,2016-12-31 18:18:30,[bbq_ckn_l],Saturday


In [183]:
pizza_types = create_csv_with_pizzas_per_day(ordered_pizzas, pizza_types, pizzas, date)
ingredients_df = csv_ingredients(pizza_types)
ingredients_df = predict(pizza_types, ingredients_df, days_difference)
print('This are the ingredients that you need to buy for the week:')
display(ingredients_df)

This are the ingredients that you need to buy for the week:


,ingredient,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Total
0,Barbecued Chicken,7.345205,5.926027,8.064384,6.501370,7.191781,8.131507,8.169863,51.330137
1,Red Peppers,50.083562,42.297260,47.753425,46.689041,49.834247,51.080822,55.865753,343.604110
2,Green Peppers,15.409589,12.926027,15.783562,14.057534,14.872603,16.541096,17.442466,107.032877
3,Tomatoes,82.091781,68.820548,79.963014,78.409589,82.302740,84.671233,91.939726,568.198630
4,Red Onions,60.094521,48.606849,59.010959,56.805479,59.806849,62.606849,64.908219,411.839726
...,...,...,...,...,...,...,...,...,...
60,Parmigiano Reggiano Cheese,5.916438,5.178082,5.791781,7.095890,7.210959,7.441096,7.249315,45.883562
61,Eggplant,2.963014,2.713699,2.090411,2.291781,2.953425,2.752055,2.905479,18.669863
62,Zucchini,7.191781,6.271233,6.367123,6.213699,7.153425,7.239726,7.747945,48.184932
63,Sun-dried Tomatoes,5.312329,4.880822,5.235616,5.494521,5.091781,5.331507,5.724658,37.071233
